<a href="https://colab.research.google.com/github/sabatn/Cancer-Prediction-from-RNA-Seq-Data/blob/main/Cancer_Prediction_from_RNA_Seq_Data_(TCGA_BRCA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

#Loading datasets
expr_file = "/content/TCGA-BRCA.star_tpm.tsv"
clinical_file = "/content/TCGA-BRCA.clinical.tsv"

expr = pd.read_csv(expr_file, sep="\t").set_index("Ensembl_ID")
clinical = pd.read_csv(clinical_file, sep="\t")

#Keep only common sample
common_samples = list(set(expr.columns).intersection(clinical['sample']))
expr = expr[common_samples]
clinical = clinical[clinical['sample'].isin(common_samples)]

expr_T = expr.T.reset_index().rename(columns={"index":"sample"})
merged = pd.merge(clinical, expr_T, on="sample")

#Features & Label
# Only numeric columns for model
X = merged.select_dtypes(include=np.number).fillna(0)
y = merged['tissue_type.samples']
le = LabelEncoder()
y = le.fit_transform(y)  # Tumor=1, Normal=0
classes = le.classes_Preprocessing
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA for dimensionality reduction
pca = PCA(n_components=50)
X_pca = pca.fit_transform(X_scaled)

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, stratify=y, random_state=42)

#Train Models
models = {
    'Logistic Regression': LogisticRegression(max_iter=500),
    'Random Forest': RandomForestClassifier(n_estimators=200, random_state=42),
    'SVM': SVC(kernel='linear', probability=True, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

y_preds = {}
metrics = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_preds[name] = y_pred
    acc = accuracy_score(y_test, y_pred)
    bal_acc = balanced_accuracy_score(y_test, y_pred)
    try:
        roc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])
    except:
        roc = roc_auc_score(y_test, (model.decision_function(X_test) - model.decision_function(X_test).min()) /
                            (model.decision_function(X_test).max() - model.decision_function(X_test).min()))
    metrics.append([acc, bal_acc, roc])

metrics_df = pd.DataFrame(metrics, index=models.keys(), columns=['Accuracy','Balanced Accuracy','ROC-AUC'])
print(metrics_df)

#Visualizations
best_model = metrics_df['Balanced Accuracy'].idxmax()

#Random Forest for Top 20 Genes (use original numeric X)
rf_full = RandomForestClassifier(n_estimators=200, random_state=42)
rf_full.fit(X, y)
top_idx = rf_full.feature_importances_.argsort()[-20:]
top_genes = X.columns[top_idx]
top_values = rf_full.feature_importances_[top_idx]

#plot confusion matrix and ROC
def plot_model(name):
    plt.figure(figsize=(6,4))
    sns.heatmap(confusion_matrix(y_test, y_preds[name]), annot=True, fmt="d",
                xticklabels=classes, yticklabels=classes,
                cmap="Greens" if name==best_model else "Blues")
    plt.title(f"{name} Confusion"); plt.xlabel("Pred"); plt.ylabel("True"); plt.show()

    plt.figure(figsize=(6,4))
    model = models[name]
    if hasattr(model, "predict_proba"):
        prob = model.predict_proba(X_test)[:,1]
    else:
        prob = (model.decision_function(X_test) - model.decision_function(X_test).min()) / \
               (model.decision_function(X_test).max() - model.decision_function(X_test).min())
    fpr, tpr, _ = roc_curve(y_test, prob)
    plt.plot(fpr, tpr, color='orange', label=f"AUC={auc(fpr,tpr):.2f}")
    plt.plot([0,1],[0,1],'k--')
    plt.title(f"{name} ROC"); plt.xlabel("FPR"); plt.ylabel("TPR"); plt.legend(); plt.show()

# Plot each model
for m in models.keys():
    plot_model(m)

#Bar chart
plt.figure(figsize=(6,4))
x = np.arange(len(metrics_df))
colors = {'Accuracy':'blue','Balanced Accuracy':'orange','ROC-AUC':'green'}

for i, metric in enumerate(metrics_df.columns):
    vals = metrics_df[metric].values

    #Highlight best model’s Balanced Accuracy in gold
    bar_colors = []
    for j, m in enumerate(metrics_df.index):
        if metric == "Balanced Accuracy" and m == best_model:
            bar_colors.append("gold")
        else:
            bar_colors.append(colors[metric])

    plt.bar(x + i*0.2, vals, width=0.2, color=bar_colors, label=metric)

    for j, v in enumerate(vals):
        plt.text(x[j]+i*0.2, v+0.02, f"{v:.2f}", ha='center', fontsize=8)

plt.xticks(x+0.2, metrics_df.index)
plt.ylim(0,1)
plt.ylabel("Score")
plt.title("Model Comparison")
plt.legend()
plt.grid(axis='y')
plt.show()


#Top 20 Genes
plt.figure(figsize=(6,5))
plt.barh(range(20), top_values, color='teal')
plt.yticks(range(20), top_genes)
plt.xlabel("Importance"); plt.title("Top 20 Genes"); plt.show()